# Data Science Capstone Notebook
## This workbook will be used for the final Coursera Capstone project for
## Data Science Specialization with Python Class
## September/October 2019

In [2]:
#Import needed modules
import pandas as pd
import numpy as np
from pandas import ExcelFile

%pip install xlrd

#Below needed to scrape URLs
import requests

     |████████████████████████████████| 112kB 21.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Install BeautifulSoup
#%pip install BeautifulSoup4
#from bs4 import BeautifulSoup
#soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
#install lxml
#%conda install lxml

In [5]:
#External source variable definitions
dallas_county_url = "https://www.dallascounty.org/about-us/cities/"
tarrant_county_url = "http://access.tarrantcounty.com/en/county/about-tarrant/incorporated-areas.html"
northtexascity_file = "City_Limits_2010_Census_DFW.xlsx"
city_name_file = "CountyCityList_2019.xlsx"


In [6]:
#Read City data into Dataframes

#Get City Names
city_df = pd.read_excel(city_name_file,header=0)
#Drop County column
city_df.drop(['County'], axis=1, inplace=True)
#Remove duplicate data rows (in case a city falls within multiple columns)
city_df.drop_duplicates(inplace=True)

#Get master list of north City names and locations
locs_df = pd.read_excel(northtexascity_file,header=0)
locs_df2 = locs_df[['NAME10', 'INTPTLAT10', 'INTPTLON10']]
locs_df2.rename(columns={'NAME10': 'City', 'INTPTLAT10': 'Lat', 'INTPTLON10': 'Lon'}, inplace=True)



/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [7]:
#Combine Searched cities with location coordinates in one dataframe
dfw_df = pd.merge(city_df, locs_df2, on = 'City')

dfw_df.head()

,City,Lat,Lon
0,Addison,32.958508,-96.836985
1,Balch Springs,32.713933,-96.618052
2,Carrollton,32.988360,-96.899770
3,Cedar Hill,32.584460,-96.958860
4,Cockrell Hill,32.738182,-96.888550


# Graph City Data

In [8]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [9]:
#Install Folium for graphing

#!conda install -c conda-forge folium=0.10.0 #Already installed
#!conda install -c conda-forge folium=0.5.0 --yes 

import folium # map rendering library

In [10]:
#Grapevine Latitude/Longitude (Central City)
latitude = 32.9341074
longitude = -97.076533


# create map of DFW using latitude and longitude values
map_dfw = folium.Map(location=[latitude, longitude], zoom_start=7)

# add markers to map
for lat, lng, city in list(zip(dfw_df['Lat'], dfw_df['Lon'], dfw_df['City'])):
    #print("Neighborhood: ", neighborhood, "/Borough: ", borough, "/Lat: ", lat, "/Long: ", lng)
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dfw)  
    
map_dfw

# Gathering Features from FourSquare

In [11]:
#Read FourSquare client credentials from file (to hide credentials - not sent to GitHub)

text_file = open("FourSquare_ClientID.txt", "r")
clientID_txt = text_file.readlines()[0].strip()
text_file = open("FourSquare_ClientSecret.txt", "r")
clientSecret_txt = text_file.readlines()[0].strip()

In [12]:

# Foursquare Credentials /To be Hidden/Deleted from posted version
CLIENT_ID = clientID_txt
CLIENT_SECRET = clientSecret_txt

VERSION = '20180605' # Foursquare API version
LIMIT = 100
RADIUS = 8000 #default 500meters is about .3 miles. 16090 # in Meters 16090meters = 10 miles


In [13]:
# Function to find list of nearby venues near a given list of neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=RADIUS):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, lat, lng) #used for debugging
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    #print("test round done")
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
#Read in previously acquired Client Ratings - stored in a CSV file
rating_data = "ClientRatings_2019.csv"
clientratings = pd.read_csv(rating_data)


In [16]:
clientratings.head()

,Features,Rating
0,Accessories Store,3
1,Advertising Agency,5
2,Airport,8
3,Airport Lounge,3
4,Airport Terminal,3


In [ ]:
#This code will read in the CSV file
#This should only be used if you don't run the next line to pull in fresh data from FourSquare
saved_data = "foursquare_data_venue_step1.csv"
dfw_venues = read_csv(saved_data)



In [15]:
# All Lines commented out below to minimize the number of calls to FourSquare - 
# the processed data was saved to a CSV file that we'll reference later in the code

##dfw_df_work = dfw_df.head(3) #Set to 5 records to limit what comes back in a test
#dfw_df_work = dfw_df #full data set

#dfw_venues = getNearbyVenues(names=dfw_df_work['City'],
#                                   latitudes=dfw_df_work['Lat'],
#                                   longitudes=dfw_df_work['Lon']
#                                  )

Addison 32.9585082 -96.8369852
Balch Springs 32.7139333 -96.6180523
Carrollton 32.9883599 -96.8997702
Cedar Hill 32.5844599 -96.9588596
Cockrell Hill 32.7381821 -96.8885498
Combine 32.5900827 -96.5171965
Coppell 32.9633064 -96.9905565
Dallas 32.794176 -96.7655033
DeSoto 32.5993495 -96.8629405
Duncanville 32.6459498 -96.9138704
Farmers Branch 32.9277892 -96.8785391
Ferris 32.5367234 -96.6755371
Garland 32.9098261 -96.6303571
Glenn Heights 32.5512998 -96.8543876
Grand Prairie 32.6841931 -97.0209951
Grapevine 32.9341074 -97.076533
Highland Park 32.8310564 -96.8012347
Hutchins 32.6433922 -96.7098325
Irving 32.8577478 -96.9700224
Lancaster 32.5958887 -96.7779537
Lewisville 33.0465513 -96.9818258
Mesquite 32.763903 -96.5924264
Mesquite 26.4024338 -98.9809382
Ovilla 32.5412765 -96.8834633
Richardson 32.9722914 -96.7080688
Rowlett 32.9159218 -96.5486729
Sachse 32.9690084 -96.5808825
Seagoville 32.6533835 -96.5450583
Sunnyvale 32.7997522 -96.5585498
University Park 32.8486235 -96.7952496
Wilmer

In [16]:
dfw_venues.shape

(6550, 7)

In [17]:
#dfw_venues.to_csv('foursquare_data_venue_step1.csv', sep = ',')

In [19]:
dfw_venues.describe(include='all')

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
count,6550,6550.000000,6550.000000,6550,6550.000000,6550.000000,6550
unique,70,NaN,NaN,2144,NaN,NaN,268
top,Mesquite,NaN,NaN,Starbucks,NaN,NaN,Mexican Restaurant
freq,124,NaN,NaN,151,NaN,NaN,434
mean,NaN,32.767496,-97.065896,NaN,32.767768,-97.061953,NaN
std,NaN,0.493278,0.343883,NaN,0.492716,0.336235,NaN
min,NaN,26.402434,-98.980938,NaN,26.364866,-99.040992,NaN
25%,NaN,32.675537,-97.336874,NaN,32.679178,-97.315589,NaN
50%,NaN,32.812873,-97.135076,NaN,32.811789,-97.115544,NaN
75%,NaN,32.922750,-96.854388,NaN,32.913021,-96.827364,NaN


In [20]:
dfw_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Addison,32.958508,-96.836985,Elite Cigar Cafe,32.954967,-96.837875,Smoke Shop
1,Addison,32.958508,-96.836985,Chuy's,32.953575,-96.835815,Mexican Restaurant
2,Addison,32.958508,-96.836985,Best Thai Signature,32.954790,-96.841320,Thai Restaurant
3,Addison,32.958508,-96.836985,Fogo de Chao Brazilian Steakhouse,32.953782,-96.837738,Churrascaria
4,Addison,32.958508,-96.836985,Taste of Chicago,32.950202,-96.839735,Pizza Place


In [ ]:
# one hot encoding
dfw_onehot = pd.get_dummies(dfw_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dfw_onehot['Neighborhood'] = dfw_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dfw_onehot.columns[-1]] + list(dfw_onehot.columns[:-1])
dfw_onehot = dfw_onehot[fixed_columns]

dfw_onehot.head()

In [ ]:
dfw_onehot.shape

# Group by City

In [ ]:
dfw_grouped = dfw_onehot.groupby('Neighborhood').mean().reset_index()
dfw_grouped

In [ ]:
#Save data from Foursquare to an Excel file for temp storage
dfw_grouped.to_csv('foursquare_data_grouped.csv', sep = ',')

#### Print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in dfw_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dfw_grouped[dfw_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')